In [31]:
import pandas as pd
import ast
import numpy as np

In [44]:
path = 'comparative.csv'
df = pd.read_csv(path, index_col=0)[['premise', 'hypothesis', 'prompt', 'rationale']]

In [45]:
df.rationale = df.rationale.astype(str)
df['split'] = df.rationale.apply(lambda x: x.lower().split('most likely answer'))
df.split = df.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [46]:
df.head(1)

,premise,hypothesis,prompt,rationale,split
0,A man in orange shorts sits on some steps in f...,A man in shorts sits on some steps.,Premise: A man in orange shorts sits on some s...,1. The most likely answer is **entailment**. T...,is **entailment**. this means that the hypoth...


In [47]:
# get correct index
df['correct_index'] = [{} for _ in range(len(df))]
for index, row in df.iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice.lower())

In [48]:
df['LLM_answer'] = df.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [49]:
df[df.LLM_answer.isna()]

,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer


In [50]:
# read paper's answer
paper = pd.read_csv('paper.csv', index_col=0)
paper

,premise,hypothesis,label,llm_label,rationale,input
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,neutral,neutral,The person could be training his horse for a c...,A person on a horse jumps over a broken down a...
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",contradiction,contradiction,The person on the horse is not at the diner.,A person on a horse jumps over a broken down a...
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",entailment,entailment,The person is outdoors because the airplane is...,A person on a horse jumps over a broken down a...
3,Children smiling and waving at camera,They are smiling at their parents,neutral,neutral,"They are smiling at the camera, not their pare...",Children smiling and waving at camera</s>They ...
4,Children smiling and waving at camera,There are children present,entailment,entailment,The children are present because they are smil...,Children smiling and waving at camera</s>There...
...,...,...,...,...,...,...
9995,A baby in a white and light purple outfit crying.,A baby is crying because it is hungry.,neutral,neutral,"The baby is crying, but it is not necessarily ...",A baby in a white and light purple outfit cryi...
9996,Baby in a purple onesie crying.,The baby is sleeping peacefully.,contradiction,contradiction,"The baby is crying, not sleeping.",Baby in a purple onesie crying.</s>The baby is...
9997,Baby in a purple onesie crying.,The child is under three years old.,entailment,entailment,The child is a baby.,Baby in a purple onesie crying.</s>The child i...
9998,Baby in a purple onesie crying.,The baby is a girl.,neutral,neutral,The baby is wearing a purple onesie.,Baby in a purple onesie crying.</s>The baby is...


In [51]:
# paper['premise'] = paper.input.str.split('\n').apply(lambda x: x[0])
paper.set_index(['premise', 'hypothesis'], inplace=True)
# paper.index = paper.index.str.strip()

In [52]:
df.set_index(['premise', 'hypothesis'], inplace=True)

In [53]:
paper['my_label'] = df['LLM_answer']

In [54]:
sum(paper.label == paper.my_label) / len(paper)

0.8656

In [43]:
df.to_csv('comparative - full.csv')

In [281]:
import pandas as pd
TYPE = 'contrastive'
temp = pd.read_csv(f'{TYPE} - full.csv')

import re
temp[~temp['rationale'].apply(lambda x: bool(re.match(r'\d+\.', x.strip())))]

,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer
2176,A young woman is drawing with a Sharpie marker.,A woman is drawing.,Premise: A young woman is drawing with a Sharp...,"For the premise ""A young woman is drawing with...",is also **entailment**. this means that the h...,{'entailment': 11},entailment
4384,"A greyhound runs in a race, bee striped jacket...",The greyhound is extremely fast.,"Premise: A greyhound runs in a race, bee strip...","For the premise ""A greyhound runs in a race, b...",is **contradiction**. this means that the hyp...,{'contradiction': 6},contradiction


In [282]:
temp.LLM_answer.value_counts()

neutral          3472
entailment       3398
contradiction    3120
Name: LLM_answer, dtype: int64

In [20]:
import pandas as pd
temp = pd.read_csv('D:\distilling-step-by-step\[API] ESNLI\paper.csv', index_col=0).drop_duplicates().reset_index(drop=True)

In [17]:
temp.rename(columns={'llm_label': 'LLM_answer'}, inplace=True)

In [25]:
temp[['premise', 'hypothesis', 'LLM_answer', 'rationale']].dropna().reset_index().to_csv('paper - full.csv')

In [29]:
# read causal full csv
causal = pd.read_csv('causal - full.csv', index_col=0)

In [30]:
causal[causal.rationale.isna()]

,Unnamed: 0,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer


In [23]:
# index of paper not in causal
temp.set_index(['premise', 'hypothesis'], inplace=True)
causal.set_index(['premise', 'hypothesis'], inplace=True)

In [24]:
temp[~temp.index.isin(causal.index)].reset_index().to_csv('error.csv')